In [1]:
from nitropulse import RismaData

In [2]:
risma = RismaData(workspace_dir='../assets')

Local directory for RISMA data: ../assets/inputs/RISMA_CSV_files


In [3]:
risma_stations = ['RISMA_MB1', 'RISMA_MB2', 
                  'RISMA_MB3', 'RISMA_MB4', 
                #   'RISMA_MB5', 'RISMA_MB6', 'RISMA_MB7', 'RISMA_MB8', 
                #   'RISMA_MB9', 'RISMA_MB10', 'RISMA_MB11', 'RISMA_MB12', 
                #   'RISMA_MB13'
                  ]
params = ['Air Temp', 'Soil temperature', 'Soil Moisture']
sensors= 'average'
depths = ['0 to 5 cm', '5 cm']

In [4]:
risma.download_risma_data(
    out_dir='../assets/inputs/RISMA_CSV_files', 
    stations=risma_stations, parameters=params, 
    sensors=sensors, depths=depths, 
    start_date='2010-01-01', end_date='2024-01-01')

The file '../assets/inputs/RISMA_CSV_files/RISMA_MB1_2010_to_2024.csv' exists. 
	Skipping download process for RISMA_MB1.
The file '../assets/inputs/RISMA_CSV_files/RISMA_MB2_2010_to_2024.csv' exists. 
	Skipping download process for RISMA_MB2.
The file '../assets/inputs/RISMA_CSV_files/RISMA_MB3_2010_to_2024.csv' exists. 
	Skipping download process for RISMA_MB3.
The file '../assets/inputs/RISMA_CSV_files/RISMA_MB4_2010_to_2024.csv' exists. 
	Skipping download process for RISMA_MB4.


In [5]:
risma_df = risma.load_df()
print(risma_df.shape)
risma_df.head()

Loading RISMA files:   0%|          | 0/4 [00:00<?, ?it/s]

(31127, 17)


,date,station,depth,op,mean_sst,mean_airt,SSM,SST,year,doy,Station,Depth (cm),bulk,Sand,Silt,Clay,Classification
0,2013-04-16,MB1,0 to 5 cm,0,0.1500,-1.52,0.170000,0.059378,2013,106,MB1,5,1.28,0.788,0.101,0.111,Sandy Loam
1,2013-04-17,MB1,0 to 5 cm,0,1.5185,-0.11,0.265556,2.236667,2013,107,MB1,5,1.28,0.788,0.101,0.111,Sandy Loam
2,2013-04-17,MB1,0 to 5 cm,1,1.5185,-0.11,0.142222,-0.281222,2013,107,MB1,5,1.28,0.788,0.101,0.111,Sandy Loam
3,2013-04-18,MB1,0 to 5 cm,0,2.0665,0.00,0.296667,3.088889,2013,108,MB1,5,1.28,0.788,0.101,0.111,Sandy Loam
4,2013-04-18,MB1,0 to 5 cm,1,2.0665,0.00,0.241111,-0.374000,2013,108,MB1,5,1.28,0.788,0.101,0.111,Sandy Loam
